<a href="https://colab.research.google.com/github/CindyKXLiu/YouthCourtAnalysis/blob/master/youthCourtAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd

#The use_cols are chosen through inspecting the file in Excel and was nessary due to file size
use_cols = ['REF_DATE', 'GEO', 'Offences', 'Age of accused', 'Sex of accused', 'Charge and case', 'Type of decision']
df= pd.read_csv("/content/drive/My Drive/Dataset/youthCrimeData.csv", usecols=use_cols, engine='python',  encoding='utf-8', error_bad_lines=False)


In [5]:
print(df.shape)
df.head()

(8848000, 7)


,REF_DATE,GEO,Offences,Age of accused,Sex of accused,Charge and case,Type of decision
0,1991/1992,Canada,Total offences,"Total, age of accused","Total, sex of accused",Total charges,Total decisions
1,1991/1992,Canada,Total offences,"Total, age of accused","Total, sex of accused",Total charges,Transferred to adult court
2,1991/1992,Canada,Total offences,"Total, age of accused","Total, sex of accused",Total charges,Guilty
3,1991/1992,Canada,Total offences,"Total, age of accused","Total, sex of accused",Total charges,Percentage guilty
4,1991/1992,Canada,Total offences,"Total, age of accused","Total, sex of accused",Total charges,Acquitted


In [7]:
print(df.columns)

Index(['REF_DATE', 'GEO', 'Offences', 'Age of accused', 'Sex of accused',
       'Charge and case', 'Type of decision'],
      dtype='object')


In [8]:
df.isnull().sum()

REF_DATE            0
GEO                 0
Offences            0
Age of accused      0
Sex of accused      0
Charge and case     0
Type of decision    0
dtype: int64

In [23]:
def get_col_unique_count(col):
  #my_sum = 0;
  print("Col: " + col)
  entries = df[col].unique()
  for entry in entries:
    print(entry + ": " + str((df[col] == entry).sum()))
    #my_sum = my_sum + (df[col] == entry).sum()
  #print("sum: " + str(my_sum))
  print("")

for col in df.columns:
  get_col_unique_count(col)

Col: REF_DATE
1991/1992: 273000
1992/1993: 273000
1993/1994: 273000
1994/1995: 273000
1995/1996: 273000
1996/1997: 273000
1997/1998: 273000
1998/1999: 273000
1999/2000: 302400
2000/2001: 316400
2001/2002: 316400
2002/2003: 322000
2003/2004: 322000
2004/2005: 350000
2005/2006: 350000
2006/2007: 355600
2007/2008: 355600
2008/2009: 355600
2009/2010: 361200
2010/2011: 372400
2011/2012: 372400
2012/2013: 372400
2013/2014: 372400
2014/2015: 372400
2015/2016: 372400
2016/2017: 361200
2017/2018: 361200
sum: 8848000

Col: GEO
Canada: 673400
Newfoundland and Labrador: 634200
Prince Edward Island: 645400
Nova Scotia: 673400
New Brunswick: 667800
Quebec: 645400
Ontario: 645400
Manitoba: 645400
Saskatchewan: 617400
Alberta: 600600
British Columbia: 667800
Yukon: 600600
Northwest Territories including Nunavut: 168000
Northwest Territories: 477400
Nunavut: 485800
sum: 8848000

Col: Offences
Total offences: 221200
Total Criminal Code: 221200
Criminal Code (without traffic): 221200
Crimes against the p

Observation from above:


*  The sum of each type of offecces is 221200
*  The sum of each type of charge and 
case is 1769600
*   The sum of each type of decision is 1264000

Further analysis needed to determine if this intentional or coincidental. In addition, there are so overlapping in GEO column for the Northwest Territories.



**Cleaning GEO column.**

*Assumptions* 

*   the crime rate for each province is equal
*   the ratio of population in each province (i.e. pop Ontatio/ pop Canada) did not change through time
* the data is already randomly distributed by year

Row with Canada entry will be randomly distributed to other provinces using currunt population ratio. Same is done for entries Northwest Territories including Nunavut.


In [0]:
for yr in df.REF_DATE:
  df_CAN = [df.GEO == 'Canada' && df.REF_DATE == yr].index
  

In [0]:

df.drop(df[df.Offences == 'Total offences' || df.Offences == 'Total Criminal Code'].index)
df.drop(df[df['Age of accused'] == 'Total, age of accused' || df['Age of accused'] == 'Age not applicable'].index)
df.drop(df[df['Sex of accused'] == 'Total, sex of accused'].index)
df.drop(df[df['Charge and case'] == 'Total charges' || df['Charge and case'] == 'Total cases'].index)


**First Analysis**
---


First I want to plot the youth crime rate by province against time. From inspecting the first 20 elements in the table below I noticed that there are some entries with their GEO being Canada, which would be problematic for plotting.

In [0]:
df.head(20)

To solve that problem. I wanted to first find out how many entries have Canada as their GEO. From below we see that only about 8.5 % of the entries are like that. Since it is a relatively large dataset dropping these entries would minimally impact the result.

In [0]:
df.GEO.unique()
(df.GEO == 'Canada').sum()
print ((df.GEO == 'Canada').sum() / df.shape[0])